## 1. Setup Project Path

Configure Python path to import project modules.

# LangChain Agents - Basic Concepts

This notebook demonstrates the fundamentals of LangChain agents with tool usage and memory management.

## Topics Covered:
1. Creating a simple agent with tools
2. Understanding stateless vs stateful conversations
3. Managing conversation memory manually
4. Building a chat loop with automatic memory management

Agents can use tools (functions) to perform actions and maintain conversation context across multiple turns.

In [ ]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

## 2. Create Agent with Tools (Stateless)

This example shows a basic agent with a weather tool. Note that without explicit memory management, the agent won't remember previous conversation context.

In [ ]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent

# Define a simple tool for the agent to use
@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    return f"Weather in {city}: 72°F, sunny"

# Initialize language models
ollamaLLM = ChatOllama(model="llama3-groq-tool-use")
gptLLM = ChatOpenAI(model_name="gpt-4", temperature=0)

# Create agent with tool and system prompt
agent = create_agent(
    model=gptLLM,
    tools=[get_weather],
    system_prompt="You are a helpful assistant with weather tools."
)

# First interaction - agent learns the name and checks weather
result = agent.invoke({
    "messages": [{"role": "user", "content": "My Name is Ron. What's the weather like in New York City?"}]
})
print(result["messages"][-1].content)

# Second interaction - WITHOUT conversation history
# Agent won't remember the name because we didn't pass previous messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "What is my Name"}]
})
print(result["messages"][-1].content)
# Expected: Agent won't know the name (stateless)

## 3. Agent with Manual Memory Management (Stateful)

By maintaining a conversation history and passing it with each request, the agent can remember context across multiple turns.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# Initialize conversation state (persists across turns)
messages = []

# Turn 1: User introduces themselves and asks about weather
result1 = agent.invoke({
    "messages": messages + [HumanMessage(content="My Name is Ron. What's the weather like in New York City?")]
})
# Add all messages from this turn to history
messages.extend(result1["messages"])
print("Turn 1:", result1["messages"][-1].content)

# Turn 2: User asks about their name
# Because we're passing the full message history, the agent remembers!
result2 = agent.invoke({
    "messages": messages + [HumanMessage(content="What is my Name?")]
})
messages.extend(result2["messages"])
print("\nTurn 2:", result2["messages"][-1].content)
# Expected: Agent remembers "Ron" from previous turn

## 4. Interactive Chat Loop

A reusable function for chatting with an agent, automatically managing conversation history.

In [ ]:
def chat_with_agent(agent, max_turns=10):
    """
    Interactive chat loop with automatic memory management.
    
    Args:
        agent: The LangChain agent to chat with
        max_turns: Maximum number of conversation turns (default: 10)
    
    The function maintains conversation history automatically,
    allowing the agent to remember context across multiple questions.
    """
    # Initialize empty message history
    messages = []
    
    print("Agent ready! Type 'quit' to exit.\n")
    
    for turn in range(max_turns):
        # Get user input
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
            
        # Combine conversation history with new user message
        input_messages = messages + [HumanMessage(content=user_input)]
        
        # Send to agent and get response
        result = agent.invoke({"messages": input_messages})
        
        # Update conversation history with all messages from this turn
        # (includes user message, any tool calls, and agent response)
        messages.extend(result["messages"])
        
        # Print only the latest assistant response
        print(f"Agent: {result['messages'][-1].content}\n")

# Usage: Start interactive chat
# Uncomment the line below to try it:
# chat_with_agent(agent)